In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
filepath = "../WeatherPy/Output/citiesweatherdata.csv"

city_data = pd.read_csv(filepath)
city_data.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hamilton,39.18,-84.53,71.6,88,90,4.70,US,1597756678
1,faya,18.39,42.45,87.8,15,40,6.93,SA,1597756678
2,atuona,-9.80,-139.03,78.4,78,5,21.12,PF,1597756678
3,vaini,-21.20,-175.20,71.6,100,90,10.29,TO,1597756679
4,griffin,33.25,-84.26,73.4,94,1,5.82,US,1597756679


In [10]:
##prepare heatmap fields
locations = city_data[['Lat', 'Long']].astype(float)
humidity = city_data['Humidity'].astype(float)
highest_humidity = city_data["Humidity"].max().astype(float)


In [11]:
#Create the heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=highest_humidity, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
##Narrow down the data
ideal_cities = city_data.loc[(city_data['Max Temp'] < 90) & 
                             (city_data['Max Temp'] > 70) & 
                             (city_data["Wind Speed"] < 7) & 
                             (city_data['Cloudiness']<10)]

print(f"Number of ideal cities: {len(ideal_cities)}")
ideal_cities


Number of ideal cities: 33


,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,griffin,33.25,-84.26,73.40,94,1,5.82,US,1597756679
32,juye,35.39,116.09,80.80,87,0,6.13,CN,1597756687
42,saint george,37.10,-113.58,77.00,15,1,3.36,US,1597756396
74,sao joao da barra,-21.64,-41.05,75.20,83,6,4.45,BR,1597756696
75,caravelas,-17.71,-39.25,79.50,62,0,6.91,BR,1597756497
116,lagunas,-5.23,-75.68,79.47,73,7,3.42,PE,1597756706
120,rosarito,32.33,-117.03,82.00,88,1,5.75,MX,1597756707
124,guerrero negro,27.98,-114.06,71.78,78,1,2.82,MX,1597756707
141,ponta do sol,32.67,-17.10,75.99,64,0,4.97,PT,1597756711
192,cozumel,20.51,-86.95,75.20,100,5,4.32,MX,1597756724


In [19]:
##Get nearby hotels
lats = ideal_cities['Lat'].tolist()
longs = ideal_cities['Long'].tolist()
best_cities = zip(lats,longs)
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_name = []

#Iterate through cities list and call the api to get hotel info
for location in best_cities:
    print()
    params = {
        'location': str(location[0]) + ',' + str(location[1]),
        'radius': 5000,
        'type': 'lodging',
        'keyword': 'hotel',
        'key': g_key
    }
    response = requests.get(base_url, params = params).json()
    try:
        hotel_name.append(response['results'][0]['name'])
    except:
        hotel_name.append('')
   
    

ideal_cities["Hotel"] = hotel_name
ideal_cities

/Users/tylerpitzer/opt/anaconda3/envs/pythondata2/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
4,griffin,33.25,-84.26,73.40,94,1,5.82,US,1597756679,Baymont by Wyndham Griffin
32,juye,35.39,116.09,80.80,87,0,6.13,CN,1597756687,Amun Morandi Hotel
42,saint george,37.10,-113.58,77.00,15,1,3.36,US,1597756396,Hilton Garden Inn St. George
74,sao joao da barra,-21.64,-41.05,75.20,83,6,4.45,BR,1597756696,OYO Pousada Mediterrâneo
75,caravelas,-17.71,-39.25,79.50,62,0,6.91,BR,1597756497,Mar Doce de Abrolhos
116,lagunas,-5.23,-75.68,79.47,73,7,3.42,PE,1597756706,Hospedaje Eco
120,rosarito,32.33,-117.03,82.00,88,1,5.75,MX,1597756707,Castillos del Mar
124,guerrero negro,27.98,-114.06,71.78,78,1,2.82,MX,1597756707,Hotel Los Caracoles
141,ponta do sol,32.67,-17.10,75.99,64,0,4.97,PT,1597756711,Palace of Zinos
192,cozumel,20.51,-86.95,75.20,100,5,4.32,MX,1597756724,Hotel Cozumel & Resort


In [20]:
len(ideal_cities)

33

In [21]:
ideal_cities["Hotel"].replace('', np.nan, inplace=True)
ideal_cities_clean = ideal_cities.dropna()
len(ideal_cities_clean)

/Users/tylerpitzer/opt/anaconda3/envs/pythondata2/lib/python3.6/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


28

In [22]:
places = ideal_cities_clean[['Lat', 'Long']].astype(float)
len(places)

28

In [23]:
##Create a heat map of the hotel data
hotel_list = []
for i, row in ideal_cities_clean.iterrows():
    hotel_list.append(f"{row['Hotel']} in {row['City']}, {row['Country']}") 

hotel_layer = gmaps.symbol_layer(places, fill_color='rgba(0,150,0,0.4)', stroke_color='rgba(0,0,150,0.4)', scale=5, info_box_content=hotel_list)

fig=gmaps.figure()
fig.add_layer(hotel_layer)  
fig




Figure(layout=FigureLayout(height='420px'))

In [24]:
##Create combined map
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))